In [1]:
import numpy as np
import tensorflow as tf
import scipy
from tensorflow import keras

from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

from scipy.optimize import minimize
from scipy.optimize import Bounds
from IPython.display import Image

In [2]:
# section 1
# Load MNIST data

(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [3]:
# section 2
# Preprocess data

X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

In [4]:
# section 3

activation_codes = {'sigmoid': 0, 'relu': 1, 'tanh': 2} # Define activation function codes

In [5]:
# section 4
# Define bounds for hyperparameters and Use activation function codes as bounds

bounds = ((4, 96), (0, 2), (5, 10))

In [7]:
# section 5
# L-BFGS-B optimizer

''' 
We implement the limited memory BFGS algorithm for optimization. 
BFGS is good for optimising nonlinear problems which are unconstrained.
The limited memory BFGS is good when the num of variables is large (about 1000).
'''

class L_BFGS_B:
    
    """
    Optimize the keras network model using L-BFGS-B algorithm.
    Attributes:
        model: optimization target model.
        samples: training samples.
        factr: convergence condition. typical values for factr are: 1e12 for low accuracy;
               1e7 for moderate accuracy; 10.0 for extremely high accuracy.
        m: maximum number of variable metric corrections used to define the limited memory matrix.
        maxls: maximum number of line search steps (per iteration).
        maxiter: maximum number of iterations.
        metris: log metrics
        progbar: progress bar
    """

    def __init__(self, model, x_train, y_train, factr = 1e7, m = 50, maxls = 50, batch_size = 128, maxiter = 5):
        """
        Args:
            model: optimization target model.
            samples: training samples.
            factr: convergence condition. typical values for factr are: 1e12 for low accuracy;
                   1e7 for moderate accuracy; 10.0 for extremely high accuracy.
            m: maximum number of variable metric corrections used to define the limited memory matrix.
            maxls: maximum number of line search steps (per iteration).
            maxiter: maximum number of iterations.
        """

        # Set attributes
        
        self.model = model
        # self.x_train = [ tf.constant(x, dtype=tf.float32) for x in x_train ]
        self.x_train = x_train

        # self.y_train = [ tf.constant(y, dtype=tf.float32) for y in y_train ]
        self.y_train = y_train

        self.factr = factr
        self.m = m
        self.maxls = maxls
        self.batch = batch_size
        self.maxiter = maxiter
        self.metrics = ['loss']
        
        # Initialize the progress bar
        
        self.progbar = tf.keras.callbacks.ProgbarLogger(
            count_mode = 'steps', stateful_metrics = self.metrics)
        self.progbar.set_params( {
            'verbose':1, 'epochs':1, 'steps':self.maxiter, 'metrics':self.metrics})

    def set_weights(self, flat_weights):
        
        """
        Set weights to the model.
        Args:
            flat_weights: flatten weights.
        """

        # Get model weights
        
        shapes = [ w.shape for w in self.model.get_weights() ]
        
        # Compute splitting indices
        
        split_ids = np.cumsum([ np.prod(shape) for shape in [0] + shapes ])
        
        # Reshape weights
        
        weights = [ flat_weights[from_id:to_id].reshape(shape)
            for from_id, to_id, shape in zip(split_ids[:-1], split_ids[1:], shapes) ]
        
        # Set weights to the model
        
        self.model.set_weights(weights)

    @tf.function
    
    def tf_evaluate(self, x, y):
        
        """
        Evaluate loss and gradients for weights as tf.Tensor.
        Args:
            x: input data.
        Returns:
            loss and gradients for weights as tf.Tensor.
        """
        with tf.GradientTape() as g:
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, self.model(x)))
        grads = g.gradient(loss, self.model.trainable_variables)
        return loss, grads

    def evaluate(self, weights):
        
        """
        Evaluate loss and gradients for weights as ndarray.
        Args:
            weights: flatten weights.
        Returns:
            loss and gradients for weights as ndarray.
        """

        # Update weights
        
        self.set_weights(weights)
        
        # Compute loss and gradients for weights
        
        batch_idx = [(i,j) for i,j in zip(range(0, len(X_train),self.batch), range(self.batch, len(X_train),self.batch))]
        
        loss_list, grad_list = [], []

        for i, j in batch_idx:
            X, Y = self.x_train[i:j, :, :], self.y_train[i:j, :]
            loss, grads = self.tf_evaluate(X, Y)

            # Convert tf.Tensor to flatten ndarray
            loss = loss.numpy().astype('float64')
            grads = np.concatenate([ g.numpy().flatten() for g in grads ]).astype('float64')

            loss_list.append(loss)
            grad_list.append(grads)

        loss = np.mean(loss_list)
        grads = np.mean(grad_list,axis=0)
        return loss, grads

    def callback(self, weights):
        
        """
        Callback that prints the progress to stdout.
        Args:
        eights: flatten weights.
        """
        
        self.progbar.on_batch_begin(0)
        loss, _ = self.evaluate(weights)
        self.progbar.on_batch_end(0, logs=dict(zip(self.metrics, [loss])))

    def fit(self):
        
        """
        Train the model using L-BFGS-B algorithm.
        """

        # Get initial weights as a flat vector
        
        initial_weights = np.concatenate(
            [ w.flatten() for w in self.model.get_weights() ])
        
        # Optimize the weight vector
        
        print('Optimizer: L-BFGS-B (maxiter={})'.format(self.maxiter))
        
        self.progbar.on_train_begin()
        self.progbar.on_epoch_begin(1)
        scipy.optimize.fmin_l_bfgs_b(func = self.evaluate, x0 = initial_weights,
            factr = self.factr, m = self.m, maxls = self.maxls, maxiter = self.maxiter,
            callback = self.callback)
        self.progbar.on_epoch_end(1)
        self.progbar.on_train_end()

In [8]:
# section 6
# Define lower-level (model parameter) optimization function with model definition

def create_model(hyperparameters):
    units, activation_code, epochs = hyperparameters
    activation = list(activation_codes.keys())[list(activation_codes.values()).index(int(activation_code))]
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(units = int(hyperparameters[0]), kernel_initializer = 'glorot_uniform', activation = activation),
        Dense(units = int(hyperparameters[0]), kernel_initializer = 'glorot_uniform', activation = activation),
        Dense(units = 10, kernel_initializer = 'glorot_uniform', activation = 'softmax')
    ])

    optimizer = Adam(learning_rate=1e-2)

    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model


In [9]:
# section 7
# Define upper-level (hyperparameter) optimization function
# It returns the validation accuracy as the objective value

def evaluate_objective(hyperparameters):    
    model = create_model(hyperparameters)
    history = model.fit(X_train, y_train, validation_split=0.2, epochs = int(hyperparameters[2]), batch_size = 128, verbose=0)
    history_dict = history.history
    score = history.history['val_accuracy'][-1]
    return -score

In [10]:
# section 8
# Optimize hyperparameters 

# x0 is initial value of hyperparameters

# 'L-BFGS-B'
x0 = np.array([4., 0., 5.])

res = minimize(evaluate_objective, x0 = x0, bounds=bounds, method='L-BFGS-B', options={'ftol': 1e-8}) 
best_hyperparameters = res.x
best_score = -res.fun

print('Best hyperparameters:', best_hyperparameters)
print('Best score:', best_score)

Best hyperparameters: [4.00000523e+00 1.13697303e-07 5.00000028e+00]
Best score: 0.8086666464805603


In [11]:
# section 9
# Train a final model (model parameters) with best hyperparameters

model = create_model(best_hyperparameters)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs = 10, batch_size=128, verbose=0)

In [12]:
# section 10
# Evaluate a final model with best hyperparameters

score = model.evaluate(X_test, y_test, verbose=0)
print('Final test accuracy:', score[1])

Final test accuracy: 0.8422999978065491


In [16]:
try:
    model = create_model(best_hyperparameters)
    model.summary()
    test = L_BFGS_B(model=model, x_train=X_train,y_train=y_train, maxiter=1000)
    score = test.model.evaluate(X_test, y_test, verbose=0)
    print('Final test accuracy:', score[1])
except Exception as e:
    print(e)

Model: "sequential_70"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_70 (Flatten)        (None, 784)               0         
                                                                 
 dense_210 (Dense)           (None, 4)                 3140      
                                                                 
 dense_211 (Dense)           (None, 4)                 20        
                                                                 
 dense_212 (Dense)           (None, 10)                50        
                                                                 
Total params: 3,210
Trainable params: 3,210
Non-trainable params: 0
_________________________________________________________________
Optimizer: L-BFGS-B (maxiter=1000)
1000/1000 [==============================] - 1621s 2s/step


Final test accuracy: 0.8806999921798706
